### 🐁 AmadeusGPT Quickstart

---


#### Notebook Overview:
1. Bootstrap the Jupyter interpreter 🛠️
2. Load secrets & config 🔑
3. Project initialization 📁
4. Run a demo query 🎬
5. Appendix 📚

#### 1 · Bootstrap the Jupyter interpreter 🛠️
-  Run the below cell *once* per fresh clone or when dependencies change
-  ⚠️ Select a 3.10 Python environment before you run the below code (if you already have the AmadeusGPT Jupyter kernel select that and skip to [Jump to “Load secrets & config”](#load-secrets-config)
)
<a id="bootstrap-jupyter-interpreter"></a>

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet -e .[dev]            # editable install → reads pyproject.toml
%pip install --quiet ipykernel       # needed for kernel registration
%matplotlib inline

In [ ]:
%pip install MoviePy --quiet # needed for video generation during behavioral analysis

Note: you may need to restart the kernel to use updated packages.


Register the interpreter as a named Jupyter kernel - unless you already have access to the AmadeusGPT (venv)

In [ ]:
!python -m ipykernel install --user --name amadeus-gpt-env --display-name "AmadeusGPT (venv)"

print("✅  Bootstrapping done – now **Restart** the kernel (Kernel ▶ Restart) and select “AmadeusGPT (venv)” in the top-right kernel picker.")

<a id="load-secrets-config"></a>
#### 2 · Load secrets & config 🔑


In [2]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path, override=True)
print("🔐  Environment loaded from:", env_path or "❌  .env file not found. Create one and add your OPENAI_API_KEY='api_key' to complete this step")


🔐  Environment loaded from: c:\Users\hugos\Desktop\AmadeusGPT\.env


#### 3 · Project initialisation 📁
You should re-run this each time you (re-)open this program.

In [1]:
import os
from amadeusgpt import AMADEUS, create_project
from amadeusgpt.utils import parse_result

# ── resolve repo root dynamically ──
ROOT = os.getcwd()

DATA_DIR   = os.path.join(ROOT, "data")
RESULT_DIR = os.path.join(ROOT, "results")

project_cfg = create_project(
    data_folder   = DATA_DIR,
    result_folder = RESULT_DIR,
    **{
        "data_info.video_suffix": ".avi",
        "keypoint_info.use_3d"  : True,
    }
)

amadeus = AMADEUS(project_cfg)
print("✅  Project ready:", project_cfg)


Project created at c:\Users\hugos\Desktop\AmadeusGPT\results. Results will be saved to c:\Users\hugos\Desktop\AmadeusGPT\results
The project will load video files (*.avi) and optionally keypoint files from c:\Users\hugos\Desktop\AmadeusGPT\data
A copy of the project config file is saved at c:\Users\hugos\Desktop\AmadeusGPT\results\config.yaml
{'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data',
               'result_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\results',
               'video_suffix': '.avi'},
 'keypoint_info': {'include_confidence': False, 'use_3d': True},
 'llm_info': {'gpt_model': 'gpt-4o',
              'keep_last_n_messages': 2,
              'max_tokens': 4096,
              'temperature': 0.0},
 'object_info': {'load_objects_from_disk': False, 'use_grid_objects': False},
 'video_info': {'scene_frame_number': 1}}
✅  Project ready: {'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data', 'result_folder': 'c:\\Users\\

#### 4 · Run a demo query 🎬


### Appendix 📚

---

In [3]:
from pathlib import Path
import json, traceback

queries_path = Path("queries.json")
with queries_path.open(encoding="utf‑8") as f:
    queries = json.load(f)

TEMPLATE = (
    "Follow the definition below *verbatim*.  "
    "Identify every **continuous bout** that satisfies the rules then return ONLY the list of (start_frame, end_frame) tuples "
    "using 0‑based indexing. Do not add explanations or any other text.\n\n"
)

for key, definition in queries.items():
    try:
        prompt = (
            TEMPLATE
            + definition
            + f"\n\nAfter processing, save the tuple list to 'results/{key}.txt'."
        )

        print(f"🔍  Querying '{key}'…")
        msg = amadeus.step(prompt)
        parse_result(amadeus, msg)

    except Exception as e:
        print(f"❌  Error processing '{key}': {e}")
        traceback.print_exc()
        continue

🔍  Querying 'locomotion'…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


doing active forgetting


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.1954 $
current input tokens 4576
current accumulated tokens 29197


...

[(2, 16), (20, 31), (203, 217), (305, 311), (714, 720), (734, 759), (766, 772), (1254, 1260), (2010, 2016), (2100, 2106), (2204, 2210), (2266, 2272), (2297, 2303), (2618, 2624), (2635, 2657), (2679, 2685), (2800, 2806), (3088, 3094), (3222, 3228), (3591, 3597), (4123, 4129), (4241, 4247), (4262, 4268), (4329, 4360), (4366, 4381), (4396, 4402), (4406, 4412), (4559, 4571), (4609, 4615), (4984, 4990), (5406, 5412), (5450, 5456), (5468, 5476), (5509, 5519), (5526, 5533), (5537, 5559), (5565, 5571), (5600, 5607), (6018, 6026), (6030, 6037), (6668, 6674), (7110, 7116), (7142, 7148), (7548, 7554), (7756, 7762), (7839, 7845), (7879, 7885), (7890, 7896), (8099, 8105), (8303, 8309), (8339, 8345), (8417, 8424), (8430, 8442), (8502, 8508), (8527, 8533), (8549, 8577), (8624, 8630), (8640, 8646), (9025, 9039), (9274, 9280), (9307, 9313), (9344, 9350), (9366, 9375), (9419, 9435), (9486, 9492), (9543, 9557), (9561, 9567), (9583, 9598), (9618, 9624), (9656, 9662), (9826, 9832), (9849, 9872), (10130, 10136), (10269, 10275), (10459, 10465), (10498, 10504), (10595, 10601), (10667, 10673)]

[(2, 16), (20, 31), (203, 217), (305, 311), (714, 720), (734, 759), (766, 772), (1254, 1260), (2010, 2016), (2100, 2106), (2204, 2210), (2266, 2272), (2297, 2303), (2618, 2624), (2635, 2657), (2679, 2685), (2800, 2806), (3088, 3094), (3222, 3228), (3591, 3597), (4123, 4129), (4241, 4247), (4262, 4268), (4329, 4360), (4366, 4381), (4396, 4402), (4406, 4412), (4559, 4571), (4609, 4615), (4984, 4990), (5406, 5412), (5450, 5456), (5468, 5476), (5509, 5519), (5526, 5533), (5537, 5559), (5565, 5571), (5600, 5607), (6018, 6026), (6030, 6037), (6668, 6674), (7110, 7116), (7142, 7148), (7548, 7554), (7756, 7762), (7839, 7845), (7879, 7885), (7890, 7896), (8099, 8105), (8303, 8309), (8339, 8345), (8417, 8424), (8430, 8442), (8502, 8508), (8527, 8533), (8549, 8577), (8624, 8630), (8640, 8646), (9025, 9039), (9274, 9280), (9307, 9313), (9344, 9350), (9366, 9375), (9419, 9435), (9486, 9492), (9543, 9557), (9561, 9567), (9583, 9598), (9618, 9624), (9656, 9662), (9826, 9832), (9849, 9872), (10130, 10136), (10269, 10275), (10459, 10465), (10498, 10504), (10595, 10601), (10667, 10673)]

[(2, 16), (20, 31), (203, 217), (305, 311), (714, 720), (734, 759), (766, 772), (1254, 1260), (2010, 2016), (2100, 2106), (2204, 2210), (2266, 2272), (2297, 2303), (2618, 2624), (2635, 2657), (2679, 2685), (2800, 2806), (3088, 3094), (3222, 3228), (3591, 3597), (4123, 4129), (4241, 4247), (4262, 4268), (4329, 4360), (4366, 4381), (4396, 4402), (4406, 4412), (4559, 4571), (4609, 4615), (4984, 4990), (5406, 5412), (5450, 5456), (5468, 5476), (5509, 5519), (5526, 5533), (5537, 5559), (5565, 5571), (5600, 5607), (6018, 6026), (6030, 6037), (6668, 6674), (7110, 7116), (7142, 7148), (7548, 7554), (7756, 7762), (7839, 7845), (7879, 7885), (7890, 7896), (8099, 8105), (8303, 8309), (8339, 8345), (8417, 8424), (8430, 8442), (8502, 8508), (8527, 8533), (8549, 8577), (8624, 8630), (8640, 8646), (9025, 9039), (9274, 9280), (9307, 9313), (9344, 9350), (9366, 9375), (9419, 9435), (9486, 9492), (9543, 9557), (9561, 9567), (9583, 9598), (9618, 9624), (9656, 9662), (9826, 9832), (9849, 9872), (10130, 10136), (10269, 10275), (10459, 10465), (10498, 10504), (10595, 10601), (10667, 10673)]

[(2, 16), (20, 31), (203, 217), (305, 311), (714, 720), (734, 759), (766, 772), (1254, 1260), (2010, 2016), (2100, 2106), (2204, 2210), (2266, 2272), (2297, 2303), (2618, 2624), (2635, 2657), (2679, 2685), (2800, 2806), (3088, 3094), (3222, 3228), (3591, 3597), (4123, 4129), (4241, 4247), (4262, 4268), (4329, 4360), (4366, 4381), (4396, 4402), (4406, 4412), (4559, 4571), (4609, 4615), (4984, 4990), (5406, 5412), (5450, 5456), (5468, 5476), (5509, 5519), (5526, 5533), (5537, 5559), (5565, 5571), (5600, 5607), (6018, 6026), (6030, 6037), (6668, 6674), (7110, 7116), (7142, 7148), (7548, 7554), (7756, 7762), (7839, 7845), (7879, 7885), (7890, 7896), (8099, 8105), (8303, 8309), (8339, 8345), (8417, 8424), (8430, 8442), (8502, 8508), (8527, 8533), (8549, 8577), (8624, 8630), (8640, 8646), (9025, 9039), (9274, 9280), (9307, 9313), (9344, 9350), (9366, 9375), (9419, 9435), (9486, 9492), (9543, 9557), (9561, 9567), (9583, 9598), (9618, 9624), (9656, 9662), (9826, 9832), (9849, 9872), (10130, 10136), (10269, 10275), (10459, 10465), (10498, 10504), (10595, 10601), (10667, 10673)]

[(2, 16), (20, 31), (203, 217), (305, 311), (714, 720), (734, 759), (766, 772), (1254, 1260), (2010, 2016), (2100, 2106), (2204, 2210), (2266, 2272), (2297, 2303), (2618, 2624), (2635, 2657), (2679, 2685), (2800, 2806), (3088, 3094), (3222, 3228), (3591, 3597), (4123, 4129), (4241, 4247), (4262, 4268), (4329, 4360), (4366, 4381), (4396, 4402), (4406, 4412), (4559, 4571), (4609, 4615), (4984, 4990), (5406, 5412), (5450, 5456), (5468, 5476), (5509, 5519), (5526, 5533), (5537, 5559), (5565, 5571), (5600, 5607), (6018, 6026), (6030, 6037), (6668, 6674), (7110, 7116), (7142, 7148), (7548, 7554), (7756, 7762), (7839, 7845), (7879, 7885), (7890, 7896), (8099, 8105), (8303, 8309), (8339, 8345), (8417, 8424), (8430, 8442), (8502, 8508), (8527, 8533), (8549, 8577), (8624, 8630), (8640, 8646), (9025, 9039), (9274, 9280), (9307, 9313), (9344, 9350), (9366, 9375), (9419, 9435), (9486, 9492), (9543, 9557), (9561, 9567), (9583, 9598), (9618, 9624), (9656, 9662), (9826, 9832), (9849, 9872), (10130, 10136), (10269, 10275), (10459, 10465), (10498, 10504), (10595, 10601), (10667, 10673)]

#### 5 · Get Video Samples 🎬



#### 1. Updating code  
- After you’ve made changes to your package source, simply re-run the dependency‐install cell. [Re-install dependencies here.](#bootstrap-jupyter-interpreter)
#### 2. Troubleshooting

* **Kernel name not visible?**

  * Reload VS Code/Notebook window (e.g. ⌘⇧P → “Developer: Reload Window”).
  * Make sure you're selecting Jupyter Kernels not Python Environments to find `AmadeusGPT (venv)`
  * Ensure the workspace interpreter is set to your `.venv` (click the Python version in the status bar).

* **Version or Downloading Issues?**
  
  * Versioning errors occur if your Kernel is not Python version 3.10 (required for AmadeusGPT).
    *  Ensure that you have this version of Python installed and have selected it as the version for your environment
    *  You can check this in top right corner. If you see it's Python 3.10... that's not the issue.

* **“Package X not found” after reinstalling**

  * Confirm you’ve restarted the kernel (Kernel ▶ Restart).
  * Run `pip list` in a cell to verify that your package is installed in the active environment.

* **Still stuck?**

  * In a terminal, run:

    ```bash
    jupyter kernelspec list
    ```

    to verify that `amadeus-gpt-env` appears under your user kernels.
  * Stop & restart the Jupyter server
    (in VS Code: ⌘⇧P → “Jupyter: Stop Jupyter Server”, then run any cell again).